In [1]:
%load_ext autoreload
%autoreload 2

### External Imports

In [2]:
import numpy
import random
import typing as t
import pathlib as pb

### Internal Imports

In [3]:
import vistraff

### Environment Setup

In [4]:
PATH_ROOT: pb.Path = pb.Path('.')
PATH_DATA: pb.Path = PATH_ROOT / 'data'
SEED: int = 787

In [5]:
numpy.random.default_rng(SEED)
numpy.random.seed(SEED)
random.seed(SEED)

### Data Loading

### Prediction

### Saving and evaluating the results